In [1]:
from enhancedAgent_v2 import *
from pythonTask import *

2025-06-11 04:18:07,587 - psutil_patch - INFO - psutil 已经导入，补丁不会再次导入它
2025-06-11 04:18:18,510 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [2]:
调用智能体的原则='''

### 智能体协同七诫
1. **能力透明**  
   *每个智能体必须显式声明能力边界*  
   > 教训：未验证方法存在导致流程崩溃

2. **状态显化**  
   *跨智能体数据必须通过受控总线传递*  
   > 教训：隐式变量共享引发不可测错误

3. **错误熔断**  
   *单点故障必须隔离在发生层*  
   > 教训：模型错误扩散至文件验证层

4. **验证前移**  
   *关键检查点嵌入执行流中部*  
   > 教训：后置验证增加回溯成本

5. **知识反哺**  
   *异常模式须转化为防范知识*  
   > 教训：相同错误在不同环节重复发生

6. **熵感控制**  
   *系统需持续监测并压缩认知混乱度*  
   > 教训：错误处理逻辑自身产生新熵

7. **量子信任**  
   *所有交互需携带可验证执行证明*  
   > 教训：依赖未经验证的输出导致连环失败

### 终极法则
**智能体不是工具而是生态**  
> 成功系统 = 清晰能力边界 × 严格状态管理 × 自愈机制  
> 失败系统 = 模糊责任 + 隐式依赖 + 脆弱传递  

这七条戒律构成智能体协同的免疫基因，使系统从机械执行进化为有机生命体。 
'''

In [3]:
# 销售分析任务
# 设置代理服务器
import os
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

# os.environ["AGENT_MAX_TOKENS"] = "1000000"
from pythonTask import *
from knowledge_agent import promgraming_knowledge

 
# llm=llm_claude_sonnet_4
llm=llm_gemini_2_5_pro_preview_06_05_google
# llm=llm_deepseek

# 实例化 MultiStepAgent_v2 时不传入 agent_specs
multi_agent = MultiStepAgent_v2(llm=llm)
multi_agent.loadKnowledge(调用智能体的原则)

# 使用 register_agent 动态注册 Agent
general_agent = Agent(llm=llm_deepseek)
general_agent.api_specification='''
general_agent,擅长执行各种任务
'''
multi_agent.register_agent(
    name="general_agent",
    instance=general_agent
)

document_agent = Agent(llm=llm_deepseek)
document_agent.loadKnowledge(promgraming_knowledge)
document_agent.loadKnowledge('如果指令要求你写文档，你应该调用gemini语言模型生成文档')
document_agent.api_specification='''
文档Agent,擅长调用gemini模型写文档
'''
multi_agent.register_agent(
    name="document_agent",
    instance=document_agent
)


# 示例主指令
# main_instruction = "请用python写一个hello world程序"
main_instruction = """

# 销售数据分析任务

sales_data.csv是销售数据文件，请使用此文件进行数据分析。

# 规则
1. 不要生成图表
2. 报告中必须包含每个地区，每个产品，每个销售人员的销售额
3. 分析报告保存到sales_analysis_report.md
4. 分析报告必须调用gemini模型生成

"""

# 执行多步骤任务
result = multi_agent.execute_multi_step(main_instruction)
print("多步骤执行结果：")
print(result)

2025-06-11 04:18:51,029 - enhancedAgent_v2 - DEBUG - 已注册 Agent: general_agent
2025-06-11 04:18:51,033 - enhancedAgent_v2 - DEBUG - 已注册 Agent: document_agent


2025-06-11 04:19:32,384 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:19:32,539 - enhancedAgent_v2 - DEBUG - 从对象中解析到步骤，共 7 个步骤
2025-06-11 04:19:32,542 - enhancedAgent_v2 - DEBUG - 生成计划: [{'id': 'info1', 'name': '确认销售数据文件存在', 'instruction': "请检查当前工作目录下是否存在名为 'sales_data.csv' 的文件。", 'agent_name': 'general_agent', 'instruction_type': 'execution', 'phase': 'information', 'expected_output': "返回文件是否存在的结果，例如：'文件 sales_data.csv 存在' 或 '文件 sales_data.csv 不存在'。", 'prerequisites': '无', 'status': 'pending'}, {'id': 'info2', 'name': '探查数据文件结构', 'instruction': "如果 'sales_data.csv' 文件存在，请使用pandas库读取文件的前5行，并列出所有列名，以了解数据结构，为后续数据分析做准备。", 'agent_name': 'general_agent', 'instruction_type': 'execution', 'phase': 'information', 'expected_output': "一个包含文件列名列表和前5行数据概览的字符串。这对于确认数据包含'地区'、'产品'、'销售人员'和'销售额'等必要字段至关重要。", 'prerequisites': "步骤 'info1' 确认 'sales_data.csv' 文件存在。", 'status': 'pending'}, {'id': 'exec1', 'name': 


当前执行计划:
[
  {
    "id": "info1",
    "name": "确认销售数据文件存在",
    "instruction": "请检查当前工作目录下是否存在名为 'sales_data.csv' 的文件。",
    "agent_name": "general_agent",
    "instruction_type": "execution",
    "phase": "information",
    "expected_output": "返回文件是否存在的结果，例如：'文件 sales_data.csv 存在' 或 '文件 sales_data.csv 不存在'。",
    "prerequisites": "无",
    "status": "pending"
  },
  {
    "id": "info2",
    "name": "探查数据文件结构",
    "instruction": "如果 'sales_data.csv' 文件存在，请使用pandas库读取文件的前5行，并列出所有列名，以了解数据结构，为后续数据分析做准备。",
    "agent_name": "general_agent",
    "instruction_type": "execution",
    "phase": "information",
    "expected_output": "一个包含文件列名列表和前5行数据概览的字符串。这对于确认数据包含'地区'、'产品'、'销售人员'和'销售额'等必要字段至关重要。",
    "prerequisites": "步骤 'info1' 确认 'sales_data.csv' 文件存在。",
    "status": "pending"
  },
  {
    "id": "exec1",
    "name": "计算核心销售统计数据",
    "instruction": "使用pandas库加载 'sales_data.csv' 文件。然后，分别按'地区'(Region), '产品'(Product), 和 '销售人员'(Salesperson)进行分组，并计算每个组的总销售额(Sales)。将这三个分析结果（每个地区、每个产品、每个销售人员的销售额）

2025-06-11 04:19:45,814 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:20:00,970 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:20:01,987 - openai._base_client - INFO - Retrying request to /chat/completions in 0.451209 seconds
2025-06-11 04:20:02,526 - openai._base_client - INFO - Retrying request to /chat/completions in 0.965126 seconds
2025-06-11 04:20:03,575 - enhancedAgent_v2 - WARNING - 判断步骤可执行性时出错: Connection error.
2025-06-11 04:20:03,691 - openai._base_client - INFO - Retrying request to /chat/completions in 0.431067 seconds
2025-06-11 04:20:04,220 - openai._base_client - INFO - Retrying request to /chat/completions in 0.841343 seconds
2025-06-11 04:20:05,120 - enhancedAgent_v2 - WARNING - 判断步骤可执行性时出错: Connection error.
2025-06-11 04:20:05,252 - openai._base_client - INFO - Retrying request to


执行步骤 1/7: 确认销售数据文件存在


...

好的，`general_agent` 已接收指令并开始执行。

为了验证报告内容的完整性，`general_agent` 在其Jupyter Notebook环境中执行了以下Python代码。

```python
import os

# 定义要验证的文件名
file_name = 'sales_analysis_report.md'

# 定义需要检查的关键内容（报告中的标题或关键词）
required_sections = [
    "按地区销售分析",
    "按产品销售分析",
    "按销售人员业绩分析"
]

# 检查文件是否存在
if not os.path.exists(file_name):
    result_message = f"验证失败：文件 '{file_name}' 不存在。"
else:
    try:
        # 读取文件内容
        with open(file_name, 'r', encoding='utf-8') as f:
            content = f.read()
        
        missing_sections = []
        # 逐一检查每个必需的部分是否存在于文件内容中
        for section in required_sections:
            if section not in content:
                missing_sections.append(section)
        
        # 根据检查结果生成最终信息
        if not missing_sections:
            result_message = "报告内容验证通过，包含所有必需的分析部分。"
        else:
            # 列出所有缺失的部分
            missing_list = ", ".join(missing_sections)
            result_message = f"验证失败：报告中缺少以下部分：{missing_list}。"

    except Exception as e:
        resu

2025-06-11 04:29:00,696 - root - INFO - === 开始评估 ===
2025-06-11 04:29:00,701 - root - INFO - 使用 1 个评估器进行评估...
2025-06-11 04:29:00,703 - root - INFO - 执行评估器 #1:
2025-06-11 04:29:12,068 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 04:29:12,090 - root - INFO - 评估器 #1 评估结果: 成功
2025-06-11 04:29:12,093 - root - INFO - === 评估总结 ===
2025-06-11 04:29:12,095 - root - INFO - 整体评估结果: 成功
2025-06-11 04:29:12,097 - root - INFO - 详细信息: 所有评估器都通过
#1: 任务是执行一个Python脚本来验证文件内容。脚本成功执行，没有返回任何错误。其标准输出'报告内容验证通过，包含所有必需的分析部分。'明确表明了验证已成功完成，并找到了所有必需的部分，这完全符合用户的指令要求。根据规则2，程序成功执行并输出了预期结果，因此任务完成。
2025-06-11 04:29:12,103 - pythonTask - INFO - 开始生成指令最终结果
2025-06-11 04:29:12,108 - pythonTask - INFO - result.success: True
2025-06-11 04:29:12,110 - pythonTask - INFO - result.code: import os

# 定义要验证的文件名
file_name = 'sales_analysis_report.md'

# 定义需要检查的关键内容（报告中的标题或关键词）
required_sections = [
    "按地区销售分析",
    "按产品销售分析",
    "按销售人员业绩

好的，任务已完成。

我已经读取并检查了 `sales_analysis_report.md` 文件的内容。

**验证结果：** 报告内容验证通过，其中包含了您指定的'各地区销售额'、'各产品销售额'和'各销售人员销售额'所有关键分析部分。

2025-06-11 04:29:33,806 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"



决策结果: complete
原因: 所有计划的步骤都已成功执行，包括最终的验证步骤。最后一步 'verify2' 确认了生成的报告文件内容完整，符合所有要求。由于没有剩余步骤且原始目标已达成，工作流现已完成。
多步骤执行结果：

## 执行摘要
- 总步骤数: 7
- 已完成: 7
- 失败: 0
- 未执行: 0


继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
决策为完成执行。



In [ ]:
print('hello world')

In [ ]:
response=multi_agent.chat_stream('写个使用智能体的简短的经验教训总结')
for chunk in response:
    print(chunk, end='', flush=True)

In [6]:
# 示例2：翻译模式（方案2）
llm = llm_deepseek

# 创建翻译模式的多步骤智能体（方案2：支持控制流）
workflow_agent = MultiStepAgent_v2(llm=llm_gemini_2_5_pro_preview_06_05_google, use_autonomous_planning=False)

# 注册智能体
coder_agent = Agent(llm=llm)
coder_agent.api_specification = '''
通用编程智能体,擅长编写python代码
'''
workflow_agent.register_agent(
    name="coder",
    instance=coder_agent
)

test_agent = Agent(llm=llm)
test_agent.api_specification = '''
软件测试智能体,擅长运行和测试python代码
'''
workflow_agent.register_agent(
    name="tester",
    instance=test_agent
)

# 翻译模式 - 用户提供详细步骤，AI翻译为JSON计划并执行
print("=== 翻译模式示例（方案2：动态控制流）===")
result = workflow_agent.execute_multi_step("""
1. coder: 实现一个简单的计算器类，包含加减乘除功能和完整单元测试
2. coder: 把代码保存到simple_calculator.py
3. tester: 运行simple_calculator.py，执行所有测试
4. decision_maker: 分析测试结果，测试通过则完成工作流，测试失败则状态转移至步骤3重新执行测试
""")

2025-06-11 17:24:58,705 - enhancedAgent_v2 - DEBUG - 已注册 Agent: coder
2025-06-11 17:24:58,730 - enhancedAgent_v2 - DEBUG - 已注册 Agent: tester
2025-06-11 17:24:58,748 - enhancedAgent_v2 - DEBUG - 已注册 Agent: decision_maker
2025-06-11 17:24:58,754 - enhancedAgent_v2 - DEBUG - 自动注册了决策智能体


=== 翻译模式示例（方案2：动态控制流）===


2025-06-11 17:25:28,514 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 17:25:29,034 - enhancedAgent_v2 - DEBUG - 从对象中解析到步骤，共 4 个步骤
2025-06-11 17:25:29,036 - enhancedAgent_v2 - DEBUG - 生成计划: [{'id': 'step1', 'name': '实现计算器类及单元测试', 'instruction': '实现一个简单的计算器类，包含加减乘除功能和完整单元测试', 'agent_name': 'coder', 'instruction_type': 'execution', 'phase': 'execution', 'expected_output': '包含计算器类和单元测试的Python代码字符串', 'prerequisites': '无', 'status': 'pending'}, {'id': 'step2', 'name': '保存代码到文件', 'instruction': '把代码保存到simple_calculator.py', 'agent_name': 'coder', 'instruction_type': 'execution', 'phase': 'execution', 'expected_output': '名为simple_calculator.py的代码文件已成功创建并包含代码', 'prerequisites': '计算器类和单元测试的代码已经实现', 'status': 'pending'}, {'id': 'step3', 'name': '运行单元测试', 'instruction': '运行simple_calculator.py，执行所有测试', 'agent_name': 'tester', 'instruction_type': 'execution', 'phase': 'verification', 'expected_output': '单元测试


当前执行计划:
[
  {
    "id": "step1",
    "name": "实现计算器类及单元测试",
    "instruction": "实现一个简单的计算器类，包含加减乘除功能和完整单元测试",
    "agent_name": "coder",
    "instruction_type": "execution",
    "phase": "execution",
    "expected_output": "包含计算器类和单元测试的Python代码字符串",
    "prerequisites": "无",
    "status": "pending"
  },
  {
    "id": "step2",
    "name": "保存代码到文件",
    "instruction": "把代码保存到simple_calculator.py",
    "agent_name": "coder",
    "instruction_type": "execution",
    "phase": "execution",
    "expected_output": "名为simple_calculator.py的代码文件已成功创建并包含代码",
    "prerequisites": "计算器类和单元测试的代码已经实现",
    "status": "pending"
  },
  {
    "id": "step3",
    "name": "运行单元测试",
    "instruction": "运行simple_calculator.py，执行所有测试",
    "agent_name": "tester",
    "instruction_type": "execution",
    "phase": "verification",
    "expected_output": "单元测试的执行结果，包括成功或失败的详细信息",
    "prerequisites": "代码已保存到simple_calculator.py文件",
    "status": "pending"
  },
  {
    "id": "step4",
    "name": "分析测试结果并决策",
    "

2025-06-11 17:25:43,402 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 17:25:54,157 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-11 17:26:05,317 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"


...


执行步骤 2/4: 保存代码到文件


2025-06-11 17:28:39,431 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"


好的，我将调用`coder`来执行文件保存任务。我将指示它把上一步骤中存储在 `calculator_code` 变量里的代码写入到 `simple_calculator.py` 文件中。

```python
import os

# 检查 calculator_code 变量是否存在
if 'calculator_code' not in globals():
    print("错误：找不到 'calculator_code' 变量。请先执行代码生成步骤。")
else:
    # 准备保存文件的指令
    instruction = f"""
    请将以下代码内容保存到名为 'simple_calculator.py' 的文件中:
    
    ```python
    {calculator_code}
    ```
    """
    
    # 调用coder执行文件保存任务
    stream = coder.execute_stream(instruction)

    # 处理流式响应
    result = None
    for chunk in stream:
        # 实时打印coder的输出
        print(chunk, end="", flush=True)
        # 保存最后一个元素，即Result对象
        result = chunk

    # 检查最终执行结果
    if result.success:
        print("\n\n任务成功完成。")
        # 额外验证文件是否真的被创建
        if os.path.exists('simple_calculator.py'):
            print("验证成功：文件 'simple_calculator.py' 已成功创建。")
        else:
            print("验证失败：文件 'simple_calculator.py' 未被创建。")
    else:
        print("\n\n任务执行失败。")
        print(f"错误信息:\n{result.stderr}")

```Thinker e

Exception ignored in: <function Group.__del__ at 0x7f3d52f7d510>
Traceback (most recent call last):
  File "/home/guci/.local/lib/python3.10/site-packages/regex/_regex_core.py", line 3035, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 